
In this question you will write Python code for processing, analyzing and understanding the social network **Reddit** (www.reddit.com). Reddit is a platform that allows users to upload posts and comment on them, and is divided in _subreddits_, often covering specific themes or areas of interest (for example, [world news](https://www.reddit.com/r/worldnews/), [ukpolitics](https://www.reddit.com/r/ukpolitics/) or [nintendo](https://www.reddit.com/r/nintendo)). You are provided with a subset of Reddit with posts from Covid-related subreddits (e.g., _CoronavirusUK_ or _NoNewNormal_), as well as randomly selected subreddits (e.g., _donaldtrump_ or _razer_).

The `csv` dataset you are provided contains one row per post, and has information about three entities: **posts**, **users** and **subreddits**. The column names are self-explanatory: columns starting with the prefix `user_` describe users, those starting with the prefix `subr_` describe subreddits, the `subreddit` column is the subreddit name, and the rest of the columns are post attributes (`author`, `posted_at`, `title` and post text - the `selftext` column-, number of comments - `num_comments`, `score`, etc.).

In this exercise, you are asked to perform a number of operations to gain insights from the data.

In [173]:
# suggested imports
import pandas as pd
from nltk.tag import pos_tag
import re
from collections import defaultdict,Counter
from nltk.stem import WordNetLemmatizer
from datetime import datetime
from tqdm import tqdm
import numpy as np
import os
tqdm.pandas()
from ast import literal_eval
# nltk imports, note that these outputs may be different if you are using colab or local jupyter notebooks
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [174]:
from urllib import request
import pandas as pd
module_url = f"https://raw.githubusercontent.com/luisespinosaanke/cmt309-portfolio/master/data_portfolio_21.csv"
df = pd.read_csv(module_url, encoding='utf-8')
#module_name = module_url.split('/')[-1]
#print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
#with request.urlopen(module_url).read().decode('utf-8') as f, open(module_name,'w') as outf:
 # a = f.read()
  #outf.write(a.decode('utf-8'))


#df = pd.read_csv('data_portfolio_21.csv')
# this fills empty cells with empty strings
#df = df.fillna('')

In [175]:
df.head()

,author,posted_at,num_comments,score,selftext,subr_created_at,subr_description,subr_faved_by,subr_numb_members,subr_numb_posts,subreddit,title,total_awards_received,upvote_ratio,user_num_posts,user_registered_at,user_upvote_ratio
0,-Howitzer-,2020-08-17 20:26:04,19,1,NaN,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,BREAKING: Trump to begin hiding in mailboxes t...,0,1.00,4661,2012-11-09,-0.658599
1,-Howitzer-,2020-07-06 17:01:48,1,3,NaN,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Joe Biden's America,0,0.67,4661,2012-11-09,-0.658599
2,-Howitzer-,2020-09-09 02:29:02,3,1,NaN,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,4 more years and we can erase his legacy for g...,0,1.00,4661,2012-11-09,-0.658599
3,-Howitzer-,2020-06-23 23:02:39,2,1,NaN,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Revelation 9:6 [Transhumanism: The New Religio...,0,1.00,4661,2012-11-09,-0.658599
4,-Howitzer-,2020-08-07 04:13:53,32,622,NaN,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,"LOOK HERE, FAT",0,0.88,4661,2012-11-09,-0.658599


## P1.1 - Text data processing

### P1.1.1 - Faved by as lists

The column `subr_faved_by` contains an array of values (names of redditors who added the subreddit to which the current post was submitted), but unfortunately they are in text format, and you would not be able to process them properly without converting them to a suitable python type. You must convert these string values to Python lists, going from

```python
'["user1", "user2" ... ]'
```

to

```python
["user1", "user2" ... ]
```

**What to implement:** Implement a function `transform_faves(df)` which takes as input the original dataframe and returns the same dataframe, but with one additional column called `subr_faved_by_as_list`, where you have the same information as in `subr_faved_by`, but as a python list instead of a string.

In [77]:
def transform_faves(df):
    # your code here
    subr_faved_by_as_list = np.empty(len(df), dtype=object)
    for x in range(0,len(df)):
        subr_faved_by_as_list[x] = literal_eval(df["subr_faved_by"][x])
    df["subr_faved_by_as_list"] = subr_faved_by_as_list
    
    return df

df = transform_faves(df)
df



,author,posted_at,num_comments,score,selftext,subr_created_at,subr_description,subr_faved_by,subr_numb_members,subr_numb_posts,subreddit,title,total_awards_received,upvote_ratio,user_num_posts,user_registered_at,user_upvote_ratio,subr_faved_by_as_list
0,-Howitzer-,2020-08-17 20:26:04,19,1,NaN,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,BREAKING: Trump to begin hiding in mailboxes t...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo..."
1,-Howitzer-,2020-07-06 17:01:48,1,3,NaN,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Joe Biden's America,0,0.67,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo..."
2,-Howitzer-,2020-09-09 02:29:02,3,1,NaN,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,4 more years and we can erase his legacy for g...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo..."
3,-Howitzer-,2020-06-23 23:02:39,2,1,NaN,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Revelation 9:6 [Transhumanism: The New Religio...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo..."
4,-Howitzer-,2020-08-07 04:13:53,32,622,NaN,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,"LOOK HERE, FAT",0,0.88,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19935,zqrwiel,2020-07-23 16:39:15,11,246,NaN,2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,playboicarti,carti why,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe..."
19936,zqrwiel,2020-12-15 11:25:07,39,1,"Then I think we might get 18 songs, outro usua...",2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,playboicarti,If uzi on track 3 and 16,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe..."
19937,zqrwiel,2020-12-27 13:57:49,15,1,He has 25songs to perform plus the additional ...,2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,playboicarti,Man carti’s concerts are gonna be long af,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe..."
19938,zqrwiel,2020-12-29 12:07:10,6,1,I got goose[***]ps just by thinking about it 😬,2009-04-13,A subreddit dedicated to the discussion of hip...,"['solex125', 'redreddington22', 'HibikiSS', 'k...",8740,630857,playboicarti,Can’t wait to see Carti going full rage mode o...,0,1.00,1883,2014-02-12,0.861626,"[solex125, redreddington22, HibikiSS, klondipe..."


### P1.1.2 - Merge titles and text bodies

All Reddit posts need to have a title, but a text body is optional. However, we want to be able to access all free text information for each post without having to look at two columns every time.

**What to implement**: A function `concat(df)` that will take as input the original dataframe and will return it with an additional column called `full_text`, which will concatenate `title` and `selftext` columns, but with the following restrictions:

- 1) Wrap the title between `<title>` and `</title>` tags.
- 2) Add a new line (`\n`) between title and selftext, but only in cases where you have both values (see instruction 4).
- 3) Wrap the selftext between `<selftext>` and `</selftext>`.
- 4) You **must not** include the tags in points (1) or (3) if the values for these columns is missing. We will consider a missing value either an empty value (empty string) or a string of only one character (e.g., an emoji). Also, the value of a `full_text` column must not end in the new line character.

In [4]:
def concat(df):
    # your code here
    for x in range(0,len(df)):
        np.where(df['title'].values != '','<title>' + df['title'].values + '</title>', '')
        np.where(df['title'].values != '','<title>' + df['title'].values + '</title>', '') + np.where(((df['title'].values != '') & (df['selftext'].values != '')), '\n' , '') +  np.where(df['selftext'].values != '','<selftext>' + str(df['selftext'].values) + '</selftext>', '')
    return df

df = concat(df)
print(df)

KeyboardInterrupt: ignored

### P1.1.3 - Enrich posts 

We would like to augment our text data with linguistic information. To this end, we will _tokenize_, apply _part-of-speech tagging_, and then we will _lower case_ all the posts.

**What to implement**: A function `enrich_posts(df)` that will take as input the original dataframe and will return it with **two** additional columns: `enriched_title` and `enriched_selftext`. These columns will contain tokenized, pos-tagged and lower cased versions of the original text. **You must implement them in this order**, because the pos tagger uses casing information.

In [7]:
def enrich_posts(df):
    df['enriched_title'] = df['title'].apply(word_tokenize).apply(pos_tag)  #used postag as it 
    tlist = [[(j.lower(),k) for (j,k) in i] for i in df['enriched_title']]  #list comp
    df['enriched_title'] = df['title']
    df['enriched_title'] = df['enriched_title'].apply(word_tokenize)
    df['enriched_title'] = df['enriched_title'].apply(pos_tag)
    return df

df = enrich_posts(df)
print(df)

           author            posted_at  num_comments  score  \
0      -Howitzer-  2020-08-17 20:26:04            19      1   
1      -Howitzer-  2020-07-06 17:01:48             1      3   
2      -Howitzer-  2020-09-09 02:29:02             3      1   
3      -Howitzer-  2020-06-23 23:02:39             2      1   
4      -Howitzer-  2020-08-07 04:13:53            32    622   
...           ...                  ...           ...    ...   
19935     zqrwiel  2020-07-23 16:39:15            11    246   
19936     zqrwiel  2020-12-15 11:25:07            39      1   
19937     zqrwiel  2020-12-27 13:57:49            15      1   
19938     zqrwiel  2020-12-29 12:07:10             6      1   
19939     zqrwiel  2021-01-21 16:47:13            23      1   

                                                selftext subr_created_at  \
0                                                    NaN      2009-04-29   
1                                                    NaN      2009-04-29   
2              

## P1.2 - Answering questions with pandas


### P1.2.1 - Users with best scores

- Find the users with the highest aggregate scores (over all their posts) for the whole dataset. You should restrict your results to only those whose aggregated score is above 10,000 points, in descending order. Your code should generate a dictionary of the form `{author:aggregated_scores ... }`.

In [129]:
# your code here
agg_score = df.groupby(["author"]).score.sum().reset_index()
score_vals = np.where(agg_score.score > 10000)
agg_score_df = agg_score.iloc[score_vals]
agg_score_df = agg_score_df.sort_values(by=['score'],ascending=False)
authors = np.asarray(agg_score_df['author'])
score = np.asarray(agg_score_df['score'])
dicts = {}
for i in range(len(authors)):
    dicts[authors[i]] = score[i]
print(dicts)

{'DaFunkJunkie': 250375, 'None': 218846, 'SUPERGUESSOUS': 211611, 'jigsawmap': 210824, 'chrisdh79': 143538, 'hildebrand_rarity': 122464, 'iSlingShlong': 118595, 'hilltopye': 81245, 'tefunka': 79560, 'OldFashionedJizz': 64398, 'JLBesq1981': 58235, 'rspix000': 57107, 'Wagamaga': 47989, 'stem12345679': 47455, 'TheJeck': 26058, 'TheGamerDanYT': 25357, 'TrumpSharted': 21154, 'NotsoPG': 18518, 'SonictheManhog': 18116, 'BlanketMage': 13677, 'NewAltWhoThis': 12771, 'kevinmrr': 11900, 'Dajakesta0624': 11613, 'apocalypticalley': 10382}


### P1.2.2 - Awarded posts

Find the number of posts that have received at least one award. Your query should return only one value.

In [124]:

# your code here
Total_post = (df['total_awards_received']!=0).sum()
print(" Total Post with atleast 1 award:" + str(Total_post))

 Total Post with atleast 1 award:119


### P1.2.3 Find Covid 

Find the name and description of all subreddits where the name starts with `Covid` or `Corona` and the description contains `covid` or `Covid` anywhere. Your code should generate a dictionary of the form#

```python
  {'Coronavirus':'Place to discuss all things COVID-related',
  ...
  }
```

In [158]:
# your code here
df_val = df.loc[(df['subreddit'].str.startswith('Covid')== True) | (df['subreddit'].str.startswith('Corona')== True)]

covid_post = df_val.loc[(df_val['subr_description'].str.contains('Covid')== True) | (df_val['subr_description'].str.contains('covid')== True)]
covid_post_subredit = np.asarray(covid_post['subreddit'])
covid_post_subr_description = np.asarray(covid_post['subr_description'])
dicts = {}

for i in range(len(covid_post_subredit)):
    dicts[covid_post_subredit[i]] = covid_post_subr_description[i]
print(dicts)


{'CoronavirusCA': 'Tracking the Coronavirus/Covid-19 outbreak in California'}


### P1.2.4 - Redditors that favorite the most

Find the users that have favorited the largest number of subreddits. You must produce a pandas dataframe with **two** columns, with the following format:

```python
     redditor	    numb_favs
0	user1           7
1	user2           6
2	user3	       5
3	user4           4
...
```

where the first column is a Redditor username and the second column is the number of distinct subreddits he/she has favorited.

In [177]:
redditor = df['author'].unique()
num_favs = np.empty(len(redditor), dtype=float)
reddi_name = np.empty(len(redditor), dtype=object)
ct = 0
for reddi in redditor:
    df_faved_subred =  df.loc[(df['subr_faved_by'].str.contains(reddi) == True )]
    val = len(df_faved_subred)
    num_favs[ct] = val
    reddi_name[ct] = reddi
    ct = ct+1

df_fav_red = list(zip(reddi_name, num_favs))
df_output = pd.DataFrame(df_fav_red,
                  columns = ['redditor', 'numb_favs']) 
df_final = df_output.sort_values(by=['numb_favs'], ascending=False)
df_final.head(5)

,redditor,numb_favs
1230,JordanCarter77,5485.0
350,Peninsula99999,5171.0
515,Zhana-Aul,5160.0
1239,Kdog8273,4079.0
290,Mcnst,3968.0
